In [1]:
import altair as alt
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, parse_graph_freq
from src.spinorama.contour import compute_contour, compute_isoline
from src.spinorama.graph import display_freq, display_contour, display_template1, display_vertical
import numpy as np

df = parse_all_speakers()

Loaded 12 speakers


In [2]:
#parse_graphs_speaker('Emotiva Airmotive 6s')

In [3]:
for k in df.keys():
  print(k)

Emotiva Airmotive 6s
Harbeth Monitor
JBL 104
KEF LS50
Kali IN-8
Micca RB42
Neumann KH 80
Pioneer SP-BS22-LR
Realistic MC-1000
Revel C52
Selah Audio RC3R
Yamaha HS5


In [4]:
neumann = df['Neumann KH 80']
for k in neumann.keys():
    print(k)

CEA2034_unmelted
CEA2034
Early Reflections_unmelted
Early Reflections
Estimated In-Room Response_unmelted
Estimated In-Room Response
Horizontal Reflections_unmelted
Horizontal Reflections
Vertical Reflections_unmelted
Vertical Reflections
SPL Horizontal_unmelted
SPL Horizontal
SPL Vertical_unmelted
SPL Vertical


In [5]:
spinorama_u = neumann['CEA2034_unmelted']
for m in spinorama_u.keys():
    print(m)
spinorama = neumann['CEA2034']
for m in spinorama.keys():
    print(m)

Freq
On Axis
Listening Window
Early Reflections
Sound Power
Sound Power DI
Early Reflections DI
DI offset
Freq
Measurements
dB


In [6]:
onaxis = spinorama.loc[spinorama['Measurements']=='On Axis']
print(onaxis)

           Freq Measurements        dB
200     20.5078      On Axis   55.7961
201     21.2402      On Axis   51.9241
202     21.9727      On Axis   53.8309
203     22.7051      On Axis   55.0830
204     23.4375      On Axis   41.6382
..          ...          ...       ...
395  17753.2000      On Axis  107.9410
396  18379.4000      On Axis  106.9740
397  19027.6000      On Axis  104.8340
398  19698.5000      On Axis  104.4100
399  19999.5000      On Axis  104.0890

[200 rows x 3 columns]


In [7]:
from src.spinorama.analysis import estimates

e0, e3, e6, delta = estimates(onaxis)

print('Estimated -3dB point at %1.fHz'% e3)
print('Estimated -6dB point at %1.fHz'% e6)
print('Estimated +/-%1.1fdB from 80hz to 20kz'%delta)


Estimated -3dB point at 57Hz
Estimated -6dB point at 54Hz
Estimated +/-3.0dB from 80hz to 20kz


In [8]:
import math
import pandas as pd
import numpy as np

spl = neumann['SPL Horizontal']

radius=1
anglelist = [a for a in range(-170,180,10)]
gridC = [(radius*math.cos(p*math.pi/180), 
          radius*math.sin(p*math.pi/180)) for p in anglelist]
gridX = [(0,g[0]) for g in gridC] 
gridX = [s for s2 in gridX for s in s2]
gridY = [(0,g[1]) for g in gridC] 
gridY = [s for s2 in gridY for s in s2]

#for g in zip(gridX, gridY):
#    print('line from %d, %d' % g)

splu = neumann['SPL Horizontal_unmelted']
# 100hz 47
#  1khz 113
# 10khz 180
def hzname(i):
    if i == 47:
        return '100 Hz'
    elif i == 113:
        return '1 kHz'
    elif i == 180:
        return '10 kHz'
    else:
        return 'error'

dbX = []
dbY = []
hz  = []
for ihz in [47, 113, 180]:
    gridZ = splu.loc[ihz][1:]
    angles = [0]
    values = {}
    dbs = []
    for a, z in zip(gridZ.index,gridZ):
        angle = 0
        if a != 'On-Axis':
            angle = int(a[:-1])
            angles.append(angle)
        dbs.append(z)

    anglemin = np.array(angles).min()/10
    # print(angles)
    # print(perm)
    # print(sorted_angles)
        
    # map in 2d
    dbsX = [db*math.cos(a*math.pi/180) for a,db in zip(angles, dbs)]
    dbsY = [db*math.sin(a*math.pi/180) for a,db in zip(angles, dbs)]

    dbsX.append(dbsX[0])
    dbsY.append(dbsY[0])
    # add to global variable
    dbX.append(dbsX)
    dbY.append(dbsY)
    hz.append([ihz for i in range(0,len(dbsX))])

dbmax = max(np.array(dbX).max(), np.array(dbY).max())
dbX = [v2/dbmax for v1 in dbX for v2 in v1]
dbY = [v2/dbmax for v1 in dbY for v2 in v1]
hz  = [hzname(i2) for i1 in hz for i2 in i1]

grid_df = pd.DataFrame({'x': gridX, 'y': gridY })
grid = alt.Chart(grid_df).mark_line(
).encode(
   alt.Latitude('x:Q'), 
   alt.Longitude('y:Q'),
   size=alt.value(1)
).project(
   type='azimuthalEquidistant',
   rotate=[0, 0, 90]   
)

dbs_df = pd.DataFrame({'x': dbX, 'y': dbY, 'hz': hz })
dbs = alt.Chart(dbs_df).mark_line(
).encode(
   alt.Latitude('x:Q'), 
   alt.Longitude('y:Q'),
   alt.Color('hz:N', sort=None),
   size=alt.value(3)
).project(
   type='azimuthalEquidistant',
   rotate=[0, 0, 90]   
)

grid+dbs

alt.LayerChart(...)

In [9]:
import math
import locale
from locale import atof
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import pandas as pd
import numpy as np
from src.spinorama.load import graph_melt

title, df = parse_graph_freq('./datas/Neumann KH 80/CEA2034.txt')
print(len(df.columns))
print(df.columns)
#print(df)
#print(df.reset_index().melt(id_vars='Freq', var_name='Measurements', value_name='dB'))
title2, df2 = parse_graph_freq('./datas/Neumann KH 80/SPL Vertical.txt')
print(len(df2.columns))
print(df2.columns)
print(df2.columns[1])
df2[['Freq', 'On-Axis']].reset_index().melt(id_vars='Freq', var_name='Measurements', value_name='dB')


8
Index(['Freq', 'On Axis', 'Listening Window', 'Early Reflections',
       'Sound Power', 'Sound Power DI', 'Early Reflections DI', 'DI offset'],
      dtype='object')
37
Index(['Freq', '-170°', '-160°', '-150°', '-140°', '-130°', '-120°', '-110°',
       '-100°', '-90°', '-80°', '-70°', '-60°', '-50°', '-40°', '-30°', '-20°',
       '-10°', 'On-Axis', '10°', '20°', '30°', '40°', '50°', '60°', '70°',
       '80°', '90°', '100°', '110°', '120°', '130°', '140°', '150°', '160°',
       '170°', '180°'],
      dtype='object')
-170°


,Freq,Measurements,dB
0,20.5078,index,1.000
1,21.2402,index,2.000
2,21.9727,index,3.000
3,22.7051,index,4.000
4,23.4375,index,5.000
...,...,...,...
395,17753.2000,On-Axis,107.941
396,18379.4000,On-Axis,106.974
397,19027.6000,On-Axis,104.834
398,19698.5000,On-Axis,104.410


In [10]:
from src.spinorama.graph import *
# print(df)
speaker = 'Neumann KH 80'
width=400 
heigth=200
spinorama = display_spinorama(df, speaker, width, heigth)
onaxis    = display_onaxis(df, speaker, width, heigth)
inroom    = display_inroom(df, speaker, width, heigth)
ereflex   = display_reflection_early(df, speaker, width, heigth)
hreflex   = display_reflection_horizontal(df, speaker, width, heigth)
vreflex   = display_reflection_vertical(df, speaker, width, heigth)
hspl      = display_spl_horizontal(df, speaker, width, heigth)
vspl      = display_spl_vertical(df, speaker, width, heigth)
hcontour  = display_contour_horizontal(df,speaker, width, heigth)
vcontour  = display_contour_vertical(df,speaker, width, heigth)

vcontour